After 

@article{giannico2018,
  title={Contributions of landscape heterogeneity within the footprint of eddy-covariance towers to flux measurements},
  author={Giannico, Vincenzo and Chen, Jiquan and Shao, Changliang and Ouyang, Zutao and John, Ranjeet and Lafortezza, Raffaele},
  journal={Agricultural and Forest Meteorology},
  volume={260},
  pages={144--153},
  year={2018},
  publisher={Elsevier}
}


Add NDVI and EVI layers in previous step

## Calculation of texture features

For EVI, NDVI and k-means class perform GLCM

Rasters of _contrast_, _dissimilarity_, and _entropy_  were calculated using a  grey level co-occurrence matrix (GLCM) filter. A filter window was passed across the image Giannico et al. \cite{giannico2018} used a 3 x 3 window.

In [1]:
import numpy as np
import xarray as xr
import rioxarray
from skimage.feature import graycomatrix, graycoprops
import os
from tqdm import tqdm

# paths
ncdf = '/media/data/NEON/TEAK/mosaic/TEAK_2019_mosaic.nc'
ncdf2 = '/media/data/NEON/TEAK/mosaic/TEAK_2019_mosaic2.nc'

# open netcdf mosaic
data = xr.open_dataset(ncdf)
km = data.kmeans_label.data


In [3]:
kernel_shape = (3, 3)
sub_arrays = np.lib.stride_tricks.sliding_window_view(km, kernel_shape)

contrast = np.empty(list(sub_arrays.shape)[:2])
dissimilarity = np.empty_like(contrast)
homogeneity = np.empty_like(contrast)


def func(kernel_data):
    glcm = graycomatrix(kernel_data,
                        distances=[1],
                        angles=[0],
                        normed=True
                        , levels=kernel_data.max() + 1)

    contrast = graycoprops(glcm, 'contrast')
    dissimilarity = graycoprops(glcm, 'dissimilarity')
    homogeneity = graycoprops(glcm,  'homogeneity')

    return contrast, dissimilarity, homogeneity


def meta_func(i, j, kernel_data):
    return((i, j), func(kernel_data))


lazy = []


for i in tqdm(range(contrast.shape[0])):
    for j in range(contrast.shape[1]):
        (contrast[i, j],
        dissimilarity[i, j],
        homogeneity[i, j]) = func(sub_arrays[i, j, :, :])

# pad the edges with no_data values to get back to original ashape
contrast = np.pad(contrast, [(1, 1), (1, 1)], mode='constant', constant_values=data.no_data_value)        
dissimilarity = np.pad(dissimilarity, [(1, 1), (1, 1)], mode='constant', constant_values=data.no_data_value)        
homogeneity = np.pad(homogeneity, [(1, 1), (1, 1)], mode='constant', constant_values=data.no_data_value)

# add the scores to the xarray
data['contrast'] = xr.DataArray(contrast, dims=['x', 'y'])
data['dissimilarity'] = xr.DataArray(dissimilarity, dims=['x', 'y'])
data['homogeneity'] = xr.DataArray(homogeneity, dims=['x', 'y'])

# write to new path ( open ncdf is locked, can't overwrite)
data.to_netcdf(ncdf2)
data.close()

# rename back to original (probably exists a better way than these last 2 steps)
os.rename(ncdf2, ncdf)

100%|██████████| 9998/9998 [6:02:14<00:00,  2.17s/it]  


In [2]:
os.rename?

Signature: os.rename(src, dst, *, src_dir_fd=None, dst_dir_fd=None)
Docstring:
Rename a file or directory.

If either src_dir_fd or dst_dir_fd is not None, it should be a file
  descriptor open to a directory, and the respective path string (src or dst)
  should be relative; the path will then be relative to that directory.
src_dir_fd and dst_dir_fd, may not be implemented on your platform.
  If they are unavailable, using them will raise a NotImplementedError.
Type:      builtin_function_or_method
